# Imports

In [102]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.mediation import Mediation

import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import pickle

# Set Directory Paths

In [15]:
# Location of the data folder
%store -r dataDir

# Directory path to load formatted gfap_ferritin_dataDir Data
%store -r gfap_ferritin_dataDir

# Directory path to save Calcualted Analysis Data
%store -r gfap_ferritin_CalData

# Loading GFAP, Ferritin Dataset - %AO

In [16]:
gfap_ferr_T = pd.read_csv(os.path.join(dataDir, 'GFAP_Ferritin', 'FTLD AO output 162024.csv'))

In [17]:
gfap_ferr_T

,AutopsyID,BlockID,Hemisphere,Region,Ferritin,GFAP,AT8,TDP43
0,1986-003,1986-003-12F,N,MFC,NaN,0.054025,NaN,NaN
1,1986-003,1986-003-12F,R,MFC,0.006197,NaN,NaN,NaN
2,1986-003,1986-003-13F,R,M1,0.024898,0.054097,NaN,NaN
3,1995-005,1995-005-04E,R,OFC,0.026164,0.051511,NaN,0.004535
4,1995-005,1995-005-08E,R,ANG,0.022526,NaN,NaN,0.002226
...,...,...,...,...,...,...,...,...
3334,2021-184,2021-184-19F,R,SMTC,NaN,NaN,NaN,0.109476
3335,2021-184,2021-184-50F,L,aINS,NaN,NaN,NaN,0.228839
3336,2021-188,2021-188-09F,R,dlPFC,NaN,NaN,NaN,0.184390
3337,2021-188,2021-188-19F,R,PC,NaN,NaN,NaN,0.003372


## Format the GFAP, Ferritin Data - %AO to desired format

### Get Tau(AT8) vs TDP43 Groups

In [39]:
# Tau
tau_T = gfap_ferr_T[gfap_ferr_T['AT8'].notnull()]

# Get rid of Hemishpere Type N
tau_T = tau_T[tau_T['Hemisphere'] != 'N']

# AutopsyID
tau_INDDIDs = tau_T['AutopsyID'].unique().tolist()

print(f'Number of Unique Tau INDDIDs is: {len(tau_INDDIDs)}')

Number of Unique Tau INDDIDs is: 76


In [40]:
tau_T

,AutopsyID,BlockID,Hemisphere,Region,Ferritin,GFAP,AT8,TDP43
5,1995-057,1995-057-03E,L,MFC,0.033260,0.040396,0.133789,NaN
6,1995-057,1995-057-04E,L,OFC,0.025673,0.017537,0.126244,NaN
43,2000-066,2000-066-03F,L,MFC,0.023268,0.033943,0.003340,NaN
44,2000-066,2000-066-04F,L,OFC,0.008781,0.036202,0.004534,NaN
46,2000-066,2000-066-11F,L,SMTC,0.003953,0.034366,0.007249,NaN
...,...,...,...,...,...,...,...,...
3263,2018-216,2018-216-27F,L,OFC,NaN,NaN,0.006361,NaN
3264,2018-216,2018-216-28F,L,V1,NaN,NaN,0.000256,NaN
3265,2018-216,2018-216-29F,L,ANG,NaN,NaN,0.007745,NaN
3266,2018-216,2018-216-30F,L,aCING,NaN,NaN,0.004663,NaN


In [41]:
# TDP
tdp_T = gfap_ferr_T[gfap_ferr_T['TDP43'].notnull()]

# Get rid of Hemishpere Type N
tdp_T = tdp_T[tdp_T['Hemisphere'] != 'N']

# AutopsyID
tdp_INDDIDs = tdp_T['AutopsyID'].unique().tolist()

print(f'Number of Unique TDP INDDIDs is: {len(tdp_INDDIDs)}')

Number of Unique TDP INDDIDs is: 136


In [42]:
tdp_T

,AutopsyID,BlockID,Hemisphere,Region,Ferritin,GFAP,AT8,TDP43
3,1995-005,1995-005-04E,R,OFC,0.026164,0.051511,NaN,0.004535
4,1995-005,1995-005-08E,R,ANG,0.022526,NaN,NaN,0.002226
11,1995-217,1995-217-05F,R,OFC,0.017214,0.037690,NaN,0.000888
13,1995-217,1995-217-08F,R,ANG,0.031052,NaN,NaN,0.001599
14,1995-217,1995-217-11F,R,aCING,0.017915,0.037010,NaN,0.003493
...,...,...,...,...,...,...,...,...
3334,2021-184,2021-184-19F,R,SMTC,NaN,NaN,NaN,0.109476
3335,2021-184,2021-184-50F,L,aINS,NaN,NaN,NaN,0.228839
3336,2021-188,2021-188-09F,R,dlPFC,NaN,NaN,NaN,0.184390
3337,2021-188,2021-188-19F,R,PC,NaN,NaN,NaN,0.003372


#### Common INDDIDs between TAU vs TDP43

In [43]:
# Convert lists to sets and find the intersection
common_elements = set(tau_INDDIDs).intersection(tdp_INDDIDs)

# Check if there are any common elements
if common_elements:
    print("Common elements:", common_elements)
else:
    print("No common elements")

Common elements: {'2009-102'}


## TAU - MFC, ANG

### MFC: TAU, Ferritin %AO

In [61]:
# Get MFC Region - TAU
tau_region_type = tau_T.groupby('Region')
tau_MFC = tau_region_type.get_group('MFC')

# Remove rows where Ferritin Value is NaN
tau_MFC = tau_MFC.dropna(subset=['Ferritin'])

# Count non NaN and non zero values for each column
non_zero_non_nan_counts = (tau_MFC != 0) & tau_MFC.notna()
counts = non_zero_non_nan_counts.sum()

print(counts)

# Rename Columns / Drop TDP43
tau_MFC = tau_MFC.drop(columns=['Region', 'TDP43'])
tau_MFC = tau_MFC.rename(columns=lambda x: x + '_MFC')

AutopsyID     51
BlockID       51
Hemisphere    51
Region        51
Ferritin      51
GFAP          31
AT8           51
TDP43          0
dtype: int64


### ANG: TAU, Ferritin %AO

In [63]:
# Get MFC Region - TAU
tau_region_type = tau_T.groupby('Region')
tau_ANG = tau_region_type.get_group('ANG')

# Remove rows where Ferritin Value is NaN
tau_ANG = tau_ANG.dropna(subset=['Ferritin'])

# Count non NaN and non zero values for each column
non_zero_non_nan_counts = (tau_ANG != 0) & tau_ANG.notna()
counts = non_zero_non_nan_counts.sum()

print(counts)

# Rename Columns / Drop TDP43
tau_ANG = tau_ANG.drop(columns=['Region', 'TDP43'])
tau_ANG = tau_ANG.rename(columns=lambda x: x + '_ANG')

AutopsyID     27
BlockID       27
Hemisphere    27
Region        27
Ferritin      27
GFAP           0
AT8           27
TDP43          0
dtype: int64


In [76]:
# Merge MFC, ANG based on AutopsyID & Hemisphere
TAU_MFC_ANG = pd.merge(tau_MFC, tau_ANG, left_on=['AutopsyID_MFC', 'Hemisphere_MFC'],  
                                         right_on=['AutopsyID_ANG', 'Hemisphere_ANG'], how='inner')

TAU_MFC_ANG = TAU_MFC_ANG.drop(columns=['AutopsyID_ANG', 'Hemisphere_ANG'])
TAU_MFC_ANG = TAU_MFC_ANG.rename(columns={'AutopsyID_MFC': 'AutopsyID',
                                          'Hemisphere_MFC': 'Hemisphere'})

In [138]:
TAU_MFC_ANG

,AutopsyID,BlockID_MFC,Hemisphere,Ferritin_MFC,GFAP_MFC,AT8_MFC,BlockID_ANG,Ferritin_ANG,GFAP_ANG,AT8_ANG
0,2005-073,2005-073-29F,R,0.015930,0.034787,0.098714,2005-073-34F,0.008880,NaN,0.070316
1,2006-027,2006-027-04F,L,0.010866,0.022061,0.010569,2006-027-09F,0.017863,NaN,0.006209
2,2008-251,2008-251-29F,L,0.028530,0.045559,0.254047,2008-251-32F,0.027772,NaN,0.310358
3,2009-102,2009-102-25F,R,0.006274,0.028020,0.000332,2009-102-28F,0.018715,NaN,0.000395
4,2013-223,2013-223-06F,L,0.020343,0.063952,0.125530,2013-223-03F,0.024613,NaN,0.046466
5,2014-011,2014-011-28F,L,0.026754,NaN,0.004994,2014-011-32F,0.027113,NaN,0.002966
6,2014-052,2014-052-26F,R,0.017498,0.036848,0.007585,2014-052-30F,0.017715,NaN,0.022633
7,2014-145,2014-145-04F,L,0.010440,0.049137,0.174298,2014-145-07F,0.012162,NaN,0.001935
8,2014-212,2014-212-25F,L,0.016773,0.031280,0.009260,2014-212-28F,0.020828,NaN,0.007330
9,2014-251,2014-251-24F,L,0.026339,0.025393,0.222864,2014-251-27F,0.020757,NaN,0.036539


### OFC: TAU, Ferritin %AO

In [111]:
# Get MFC Region - TAU
tau_region_type = tau_T.groupby('Region')
tau_OFC = tau_region_type.get_group('OFC')

# Remove rows where Ferritin Value is NaN
tau_OFC = tau_OFC.dropna(subset=['Ferritin'])

# Count non NaN and non zero values for each column
non_zero_non_nan_counts = (tau_OFC != 0) & tau_OFC.notna()
counts = non_zero_non_nan_counts.sum()

print(counts)

# Rename Columns / Drop TDP43
tau_OFC = tau_OFC.drop(columns=['Region', 'TDP43'])
tau_OFC = tau_OFC.rename(columns=lambda x: x + '_OFC')

AutopsyID     43
BlockID       43
Hemisphere    43
Region        43
Ferritin      43
GFAP          24
AT8           43
TDP43          0
dtype: int64


### ANG: TAU, Ferritin %AO

In [112]:
# Get MFC Region - TAU
tau_region_type = tau_T.groupby('Region')
tau_ANG = tau_region_type.get_group('ANG')

# Remove rows where Ferritin Value is NaN
tau_ANG = tau_ANG.dropna(subset=['Ferritin'])

# Count non NaN and non zero values for each column
non_zero_non_nan_counts = (tau_ANG != 0) & tau_ANG.notna()
counts = non_zero_non_nan_counts.sum()

print(counts)

# Rename Columns / Drop TDP43
tau_ANG = tau_ANG.drop(columns=['Region', 'TDP43'])
tau_ANG = tau_ANG.rename(columns=lambda x: x + '_ANG')

AutopsyID     27
BlockID       27
Hemisphere    27
Region        27
Ferritin      27
GFAP           0
AT8           27
TDP43          0
dtype: int64


In [113]:
# Merge MFC, ANG based on AutopsyID & Hemisphere
TAU_OFC_ANG = pd.merge(tau_OFC, tau_ANG, left_on=['AutopsyID_OFC', 'Hemisphere_OFC'],  
                                         right_on=['AutopsyID_ANG', 'Hemisphere_ANG'], how='inner')

TAU_OFC_ANG = TAU_OFC_ANG.drop(columns=['AutopsyID_ANG', 'Hemisphere_ANG'])
TAU_OFC_ANG = TAU_OFC_ANG.rename(columns={'AutopsyID_OFC': 'AutopsyID',
                                          'Hemisphere_OFC': 'Hemisphere'})

In [115]:
TAU_OFC_ANG

,AutopsyID,BlockID_OFC,Hemisphere,Ferritin_OFC,GFAP_OFC,AT8_OFC,BlockID_ANG,Ferritin_ANG,GFAP_ANG,AT8_ANG
0,2007-003,2007-003-25F,R,0.011096,0.034525,0.065233,2007-003-27F,0.014662,NaN,0.238042
1,2008-251,2008-251-55F,L,0.025272,0.055223,0.136122,2008-251-32F,0.027772,NaN,0.310358
2,2010-015,2010-015-32F,R,0.025353,0.075226,0.027926,2010-015-34F,0.021949,NaN,0.011312
3,2014-011,2014-011-30F,L,0.018053,NaN,0.001812,2014-011-32F,0.027113,NaN,0.002966
4,2014-212,2014-212-26F,L,0.015978,0.031675,0.003125,2014-212-28F,0.020828,NaN,0.007330
5,2014-251,2014-251-25F,L,0.019426,NaN,0.037876,2014-251-27F,0.020757,NaN,0.036539
6,2016-220,2016-220-27F,R,0.015978,NaN,0.001000,2016-220-29F,0.018277,NaN,0.000706
7,2017-109,2017-109-26F,L,0.014577,0.038298,0.535804,2017-109-28F,0.016687,NaN,0.030775
8,2017-148,2017-148-27F,R,0.022327,NaN,0.025033,2017-148-29F,0.019824,NaN,0.006770
9,2017-158,2017-158-07F,R,0.008581,0.027740,0.067109,2017-158-04F,0.030067,NaN,0.159239


## TDP - MFC, ANG

### MFC: TDP, Ferritin %AO

In [78]:
# Get MFC Region - TAU
tdp_region_type = tdp_T.groupby('Region')
tdp_MFC = tdp_region_type.get_group('MFC')

# Remove rows where Ferritin Value is NaN
tdp_MFC = tdp_MFC.dropna(subset=['Ferritin'])

# Count non NaN and non zero values for each column
non_zero_non_nan_counts = (tdp_MFC != 0) & tdp_MFC.notna()
counts = non_zero_non_nan_counts.sum()

print(counts)

# Rename Columns / Drop TDP43
tdp_MFC = tdp_MFC.drop(columns=['Region', 'AT8'])
tdp_MFC = tdp_MFC.rename(columns=lambda x: x + '_MFC')

AutopsyID     71
BlockID       71
Hemisphere    71
Region        71
Ferritin      71
GFAP          69
AT8            0
TDP43         71
dtype: int64


### ANG: TDP, Ferritin %AO

In [79]:
# Get MFC Region - TAU
tdp_region_type = tdp_T.groupby('Region')
tdp_ANG = tdp_region_type.get_group('ANG')

# Remove rows where Ferritin Value is NaN
tdp_ANG = tdp_ANG.dropna(subset=['Ferritin'])

# Count non NaN and non zero values for each column
non_zero_non_nan_counts = (tdp_ANG != 0) & tdp_ANG.notna()
counts = non_zero_non_nan_counts.sum()

print(counts)

# Rename Columns / Drop TDP43
tdp_ANG = tdp_ANG.drop(columns=['Region', 'AT8'])
tdp_ANG = tdp_ANG.rename(columns=lambda x: x + '_ANG')

AutopsyID     69
BlockID       69
Hemisphere    69
Region        69
Ferritin      69
GFAP          28
AT8            0
TDP43         69
dtype: int64


In [80]:
# Merge MFC, ANG based on AutopsyID & Hemisphere
TDP_MFC_ANG = pd.merge(tdp_MFC, tdp_ANG, left_on=['AutopsyID_MFC', 'Hemisphere_MFC'],  
                                         right_on=['AutopsyID_ANG', 'Hemisphere_ANG'], how='inner')

TDP_MFC_ANG = TDP_MFC_ANG.drop(columns=['AutopsyID_ANG', 'Hemisphere_ANG'])
TDP_MFC_ANG = TDP_MFC_ANG.rename(columns={'AutopsyID_MFC': 'AutopsyID',
                                          'Hemisphere_MFC': 'Hemisphere'})

In [81]:
TDP_MFC_ANG

,AutopsyID,BlockID_MFC,Hemisphere,Ferritin_MFC,GFAP_MFC,TDP43_MFC,BlockID_ANG,Ferritin_ANG,GFAP_ANG,TDP43_ANG
0,1999-224,1999-224-01E,L,0.042623,0.042827,0.002274,1999-224-35F,0.009827,NaN,0.001056
1,2002-238,2002-238-15F,L,0.024325,0.034106,0.000813,2002-238-18F,0.017602,NaN,0.004022
2,2004-158,2004-158-26F,L,0.019065,0.030841,0.001851,2004-158-31F,0.015413,NaN,0.001289
3,2004-262,2004-262-26F,L,0.019302,0.022704,0.000424,2004-262-31F,0.022496,NaN,0.000579
4,2005-208,2005-208-24F,L,0.009474,0.046669,0.000646,2005-208-27F,0.013243,NaN,0.000379
5,2008-001,2008-001-23F,R,0.015520,0.026493,0.000076,2008-001-26F,0.020279,NaN,0.000002
6,2008-156,2008-156-32F,R,0.012718,0.018712,0.000031,2008-156-07E,0.007257,NaN,0.000130
7,2008-205,2008-205-30F,L,0.023678,0.053424,0.000008,2008-205-34F,0.019702,NaN,0.000010
8,2009-168,2009-168-27F,R,0.016023,0.028495,0.000403,2009-168-30F,0.019133,NaN,0.000057
9,2011-191,2011-191-37F,L,0.021863,0.027322,0.000112,2011-191-38F,0.022917,NaN,0.001514


### OFC: TDP, Ferritin %AO

In [116]:
# Get MFC Region - TAU
tdp_region_type = tdp_T.groupby('Region')
tdp_OFC = tdp_region_type.get_group('OFC')

# Remove rows where Ferritin Value is NaN
tdp_OFC = tdp_OFC.dropna(subset=['Ferritin'])

# Count non NaN and non zero values for each column
non_zero_non_nan_counts = (tdp_OFC != 0) & tdp_OFC.notna()
counts = non_zero_non_nan_counts.sum()

print(counts)

# Rename Columns / Drop TDP43
tdp_OFC = tdp_OFC.drop(columns=['Region', 'AT8'])
tdp_OFC = tdp_OFC.rename(columns=lambda x: x + '_OFC')

AutopsyID     87
BlockID       87
Hemisphere    87
Region        87
Ferritin      87
GFAP          81
AT8            0
TDP43         87
dtype: int64


### ANG: TDP, Ferritin %AO

In [117]:
# Get MFC Region - TAU
tdp_region_type = tdp_T.groupby('Region')
tdp_ANG = tdp_region_type.get_group('ANG')

# Remove rows where Ferritin Value is NaN
tdp_ANG = tdp_ANG.dropna(subset=['Ferritin'])

# Count non NaN and non zero values for each column
non_zero_non_nan_counts = (tdp_ANG != 0) & tdp_ANG.notna()
counts = non_zero_non_nan_counts.sum()

print(counts)

# Rename Columns / Drop TDP43
tdp_ANG = tdp_ANG.drop(columns=['Region', 'AT8'])
tdp_ANG = tdp_ANG.rename(columns=lambda x: x + '_ANG')

AutopsyID     69
BlockID       69
Hemisphere    69
Region        69
Ferritin      69
GFAP          28
AT8            0
TDP43         69
dtype: int64


In [118]:
# Merge MFC, ANG based on AutopsyID & Hemisphere
TDP_OFC_ANG = pd.merge(tdp_OFC, tdp_ANG, left_on=['AutopsyID_OFC', 'Hemisphere_OFC'],  
                                         right_on=['AutopsyID_ANG', 'Hemisphere_ANG'], how='inner')

TDP_OFC_ANG = TDP_OFC_ANG.drop(columns=['AutopsyID_ANG', 'Hemisphere_ANG'])
TDP_OFC_ANG = TDP_OFC_ANG.rename(columns={'AutopsyID_OFC': 'AutopsyID',
                                          'Hemisphere_OFC': 'Hemisphere'})

In [119]:
TDP_OFC_ANG

,AutopsyID,BlockID_OFC,Hemisphere,Ferritin_OFC,GFAP_OFC,TDP43_OFC,BlockID_ANG,Ferritin_ANG,GFAP_ANG,TDP43_ANG
0,1995-005,1995-005-04E,R,0.026164,0.051511,0.004535,1995-005-08E,0.022526,NaN,0.002226
1,1995-217,1995-217-05F,R,0.017214,0.037690,0.000888,1995-217-08F,0.031052,NaN,0.001599
2,1999-170,1999-170-03E,L,0.027969,0.036721,0.002450,1999-170-06E,0.011858,NaN,0.001215
3,1999-224,1999-224-31F,L,0.013363,0.042514,0.000669,1999-224-35F,0.009827,NaN,0.001056
4,2000-149,2000-149-40F,R,0.016860,0.034996,0.005378,2000-149-42F,0.019514,NaN,0.008545
5,2004-008,2004-008-05F,L,0.024742,0.047877,0.001934,2004-008-13F,0.021446,NaN,0.000265
6,2008-001,2008-001-24F,R,0.007760,0.030444,0.000063,2008-001-26F,0.020279,NaN,0.000002
7,2008-147,2008-147-06E,L,0.020561,0.040145,0.003692,2008-147-36F,0.024599,NaN,0.002295
8,2008-156,2008-156-33F,R,0.013347,0.031664,0.000484,2008-156-07E,0.007257,NaN,0.000130
9,2011-191,2011-191-06E,L,0.021249,0.063787,0.002581,2011-191-38F,0.022917,NaN,0.001514


# Mediation Analysis

## AT8_MFC $\rightarrow$ Ferritin_MFC $\rightarrow$ AT8_ANG

In [109]:
# Model for the mediator
mediator_model = sm.OLS(TAU_MFC_ANG['Ferritin_MFC'], sm.add_constant(TAU_MFC_ANG['AT8_MFC']))

# Model for the dependent variable
outcome_model = sm.OLS(TAU_MFC_ANG['AT8_ANG'], sm.add_constant(TAU_MFC_ANG[['AT8_MFC', 'Ferritin_MFC']]))

# Create the mediation analysis model / AT8_MFC: exposure = Independent Variable, Ferritin_MFC: Mediator
med_analysis = Mediation(outcome_model, mediator_model, 'AT8_MFC', 'Ferritin_MFC')

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),0.007608,-0.107346,0.116995,0.856
ACME (treated),0.007608,-0.107346,0.116995,0.856
ADE (control),0.749100,0.487099,1.027075,0.000
ADE (treated),0.749100,0.487099,1.027075,0.000
Total effect,0.756708,0.494354,1.024699,0.000
Prop. mediated (control),0.006115,-0.151094,0.154070,0.856
Prop. mediated (treated),0.006115,-0.151094,0.154070,0.856
ACME (average),0.007608,-0.107346,0.116995,0.856
ADE (average),0.749100,0.487099,1.027075,0.000
Prop. mediated (average),0.006115,-0.151094,0.154070,0.856


In [128]:
from statsmodels.formula.api import ols
# Step 1: Regress the DV on the IV
model_1 = ols('AT8_ANG ~ AT8_MFC', data=TAU_MFC_ANG).fit()
a = model_1.params['AT8_MFC']  # Use the name of the IV here, not the DV

# Step 2: Regress the MV on the IV
model_2 = ols('Ferritin_MFC ~ AT8_MFC', data=TAU_MFC_ANG).fit()
b = model_2.params['AT8_MFC']  # Again, use the IV's name

# Step 3: Regress the DV on both the IV and MV
model_3 = ols('AT8_ANG ~ AT8_MFC + Ferritin_MFC', data=TAU_MFC_ANG).fit()
c_prime = model_3.params['AT8_MFC']  # And again, use the IV's name

# The parameter for the mediator in the third model would be:
b_prime = model_3.params['Ferritin_MFC']

In [137]:
# print(a) # b1: Y = b0 + b1*X
# print(b) # b2: M = b0 + b2*M
# print(c_prime) #b4: Y = b0 + b4*X + b3*M
# print(b_prime) #b3

# ACME
print(f'ACME: {a - c_prime} or {b * b_prime}')
# ADE
print(f'ADE: {c_prime}')
# Total Effect
print(f'Total Effect: {a}')


ACME: 0.00837172523727836 or 0.00837172523727806
ADE: 0.751089145579871
Total Effect: 0.7594608708171494


## AT8_OFC $\rightarrow$ Ferritin_OFC $\rightarrow$ AT8_ANG

In [121]:
# Model for the mediator
mediator_model = sm.OLS(TAU_OFC_ANG['Ferritin_OFC'], sm.add_constant(TAU_OFC_ANG['AT8_OFC']))

# Model for the dependent variable
outcome_model = sm.OLS(TAU_OFC_ANG['AT8_ANG'], sm.add_constant(TAU_OFC_ANG[['AT8_OFC', 'Ferritin_OFC']]))

# Create the mediation analysis model
med_analysis = Mediation(outcome_model, mediator_model, 'AT8_OFC', 'Ferritin_OFC')

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),0.015248,-0.171463,0.242175,0.864
ACME (treated),0.015248,-0.171463,0.242175,0.864
ADE (control),0.061284,-0.404487,0.531197,0.790
ADE (treated),0.061284,-0.404487,0.531197,0.790
Total effect,0.076532,-0.421192,0.575241,0.740
Prop. mediated (control),0.043209,-3.173448,5.271343,0.808
Prop. mediated (treated),0.043209,-3.173448,5.271343,0.808
ACME (average),0.015248,-0.171463,0.242175,0.864
ADE (average),0.061284,-0.404487,0.531197,0.790
Prop. mediated (average),0.043209,-3.173448,5.271343,0.808


## TDP43_MFC $\rightarrow$ Ferritin_MFC $\rightarrow$ TDP43_ANG

In [110]:
# Define the mediation model
# Model for the mediator
mediator_model = sm.OLS(TDP_MFC_ANG['Ferritin_MFC'], sm.add_constant(TDP_MFC_ANG['TDP43_MFC']))

# Model for the dependent variable
outcome_model = sm.OLS(TDP_MFC_ANG['TDP43_ANG'], sm.add_constant(TDP_MFC_ANG[['TDP43_MFC', 'Ferritin_MFC']]))

# Create the mediation analysis model
med_analysis = Mediation(outcome_model, mediator_model, 'TDP43_MFC', 'Ferritin_MFC')

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),-0.002449,-0.022040,0.012351,0.766
ACME (treated),-0.002449,-0.022040,0.012351,0.766
ADE (control),0.232704,0.162528,0.302606,0.000
ADE (treated),0.232704,0.162528,0.302606,0.000
Total effect,0.230255,0.157388,0.301664,0.000
Prop. mediated (control),-0.004576,-0.104208,0.052044,0.766
Prop. mediated (treated),-0.004576,-0.104208,0.052044,0.766
ACME (average),-0.002449,-0.022040,0.012351,0.766
ADE (average),0.232704,0.162528,0.302606,0.000
Prop. mediated (average),-0.004576,-0.104208,0.052044,0.766


## TDP43_MFC $\rightarrow$ Ferritin_MFC $\rightarrow$ TDP43_ANG

In [122]:
# Define the mediation model
# Model for the mediator
mediator_model = sm.OLS(TDP_OFC_ANG['Ferritin_OFC'], sm.add_constant(TDP_OFC_ANG['TDP43_OFC']))

# Model for the dependent variable
outcome_model = sm.OLS(TDP_OFC_ANG['TDP43_ANG'], sm.add_constant(TDP_OFC_ANG[['TDP43_OFC', 'Ferritin_OFC']]))

# Create the mediation analysis model
med_analysis = Mediation(outcome_model, mediator_model, 'TDP43_OFC', 'Ferritin_OFC')

# Fit the model with bootstrapping
boot_results = med_analysis.fit(n_rep=1000)  # Using 1000 bootstrap samples

boot_results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),0.023930,-0.063882,0.143474,0.626
ACME (treated),0.023930,-0.063882,0.143474,0.626
ADE (control),0.415849,0.114530,0.744669,0.008
ADE (treated),0.415849,0.114530,0.744669,0.008
Total effect,0.439779,0.137389,0.768874,0.008
Prop. mediated (control),0.036788,-0.191051,0.376201,0.626
Prop. mediated (treated),0.036788,-0.191051,0.376201,0.626
ACME (average),0.023930,-0.063882,0.143474,0.626
ADE (average),0.415849,0.114530,0.744669,0.008
Prop. mediated (average),0.036788,-0.191051,0.376201,0.626
